In [8]:
import pandas as pd

# Load the data

df = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/user_order_final_onesheet.xlsx')


# gpt suggestions for user_order_final

In [ ]:

# Convert 'created_at' to datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# 1. Payment Method Analysis
payment_method_dist = df['payment_getway_id'].value_counts()
success_rate_by_payment = df[df['is_error'] == 0]['payment_getway_id'].value_counts() / df['payment_getway_id'].value_counts()

# 2. Order Success Analysis
order_success_rate = df['is_error'].value_counts(normalize=True)
success_rate_by_registration = df[df['is_error'] == 0]['is_user'].value_counts() / df['is_user'].value_counts()

# 3. Customer Registration Analysis
registered_vs_unregistered = df['is_user'].value_counts()
repeat_order_rate = df[df['is_user'] == 1]['unique_name'].value_counts().mean()

# 4. Time-based Analysis
orders_over_time = df.set_index('created_at').resample('M').size()
average_interval = df['interval'].mean()

# 5. Segmentation Analysis
# Apply the same RFM analysis as before, adjusting based on this dataset

# 6. Error Analysis
error_rate_by_payment = df[df['is_error'] == 1]['payment_getway_id'].value_counts() / df['payment_getway_id'].value_counts()
error_rate_by_time = df.set_index('created_at').resample('H')['is_error'].mean()

# Visualize or export the results as needed


### Customer Segmentation using Clustering

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Select relevant features
features = df[['interval', 'payment_getway_id', 'is_user', 'is_error']]

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Apply K-Means clustering
kmeans = KMeans(n_clusters=5, random_state=42)
df_new['cluster'] = kmeans.fit_predict(scaled_features)

# Analyze clusters
cluster_analysis = df_new.groupby('cluster').mean()
print("Cluster Analysis:\n", cluster_analysis)


#### Customer Lifetime Value (CLV) Prediction:

In [ ]:
import numpy as np

# Calculate total revenue for each customer
df_new['revenue'] = df_new['interval'] * df_new['payment_getway_id']
customer_revenue = df_new.groupby('unique_name')['revenue'].sum()

# Use RFM metrics and additional features for prediction
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Prepare the data
X = df_new[['R_Score', 'F_Score', 'is_user', 'payment_getway_id']]
y = customer_revenue

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict CLV
y_pred = model.predict(X_test)
print("CLV Prediction:\n", y_pred)


### Churn Prediction:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define churn as customers who haven't ordered in a long time
df_new['churn'] = (df_new['Recency'] > 90).astype(int)

# Prepare the data
X = df_new[['R_Score', 'F_Score', 'is_user', 'payment_getway_id']]
y = df_new['churn']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict churn
y_pred = model.predict(X_test)
print("Churn Prediction:\n", y_pred)


### Basket Analysis:

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# One-hot encode the item purchases
basket = df_new.groupby(['order_id', 'item_name'])['item_name'].count().unstack().fillna(0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# Apply Apriori algorithm
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print("Association Rules:\n", rules)


### Time Series Forecasting:

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Resample orders over time
orders_over_time = df_new.set_index('created_at').resample('D')['order_id'].count()

# Fit the model
model = ExponentialSmoothing(orders_over_time, trend='add', seasonal='add', seasonal_periods=12)
fit = model.fit()

# Forecast future orders
forecast = fit.forecast(30)
print("Order Forecast:\n", forecast)


### Cohort Analysis:

In [ ]:
df_new['order_month'] = df_new['created_at'].dt.to_period('M')
df_new['cohort'] = df_new.groupby('unique_name')['order_month'].transform('min')

cohort_data = df_new.groupby(['cohort', 'order_month']).size().unstack().fillna(0)
cohort_sizes = cohort_data.iloc[:, 0]
retention_matrix = cohort_data.divide(cohort_sizes, axis=0)
print("Cohort Analysis:\n", retention_matrix)


### Cluster Visualization with PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Select relevant features
features = df_new[['interval', 'payment_getway_id', 'is_user', 'is_error']]

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


In [ ]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)

# Create a DataFrame with the principal components
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['cluster'] = df_new['cluster']


In [ ]:
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Customer Segments (PCA)')
plt.colorbar(scatter, label='Cluster')
plt.show()


### radar charts

In [ ]:
segment_averages = df_new.groupby('cluster').mean()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi

# Function to plot radar chart
def plot_radar_chart(data, categories, title):
    N = len(categories)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    ax = plt.subplot(111, polar=True)
    plt.xticks(angles[:-1], categories)
    ax.set_rlabel_position(0)
    plt.yticks([0.2, 0.4, 0.6, 0.8], ["0.2", "0.4", "0.6", "0.8"], color="grey", size=7)
    plt.ylim(0, 1)

    for i in range(len(data)):
        values = data.iloc[i].tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=f'Cluster {i}')
        ax.fill(angles, values, alpha=0.25)

    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title(title)
    plt.show()

# Select relevant features for the radar chart
radar_data = segment_averages[['interval', 'payment_getway_id', 'is_user', 'is_error']]
radar_data = StandardScaler().fit_transform(radar_data)
radar_data = pd.DataFrame(radar_data, columns=['interval', 'payment_getway_id', 'is_user', 'is_error'])

plot_radar_chart(radar_data, radar_data.columns, 'Customer Segments Comparison')


### Complete Code for PCA Scatter Plot

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Load the Excel file
file_path = 'path_to_your_file/user_order_final_onesheet.xlsx'
df_new = pd.read_excel(file_path, sheet_name='Sheet1')

# Convert necessary columns to appropriate data types
df_new['created_at'] = pd.to_datetime(df_new['created_at'], errors='coerce')
df_new['last_buy'] = pd.to_datetime(df_new['last_buy'], errors='coerce')

# Select relevant features for clustering
features = df_new[['interval', 'payment_getway_id', 'is_user', 'is_error']]

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Apply PCA to reduce the dimensionality
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)

# Create a DataFrame with the principal components
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# Perform KMeans clustering for demonstration
kmeans = KMeans(n_clusters=5, random_state=42)
df_new['cluster'] = kmeans.fit_predict(scaled_features)
pca_df['cluster'] = df_new['cluster']

# Plot the PCA scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['cluster'], cmap='viridis', alpha=0.6)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Customer Segments (PCA)')
plt.colorbar(scatter, label='Cluster')
plt.show()
